### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [119]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = r"C:\Users\bendgame\Desktop\MINSTPDATA\MINSTP201902DATA4-master\Homework\04 - Pandas\HeroesOfPymoli\Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
df_pd = pd.read_csv(file_to_load)
df_pd.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [130]:
player_count = df_pd["SN"].nunique()



print(f"Total number of unique players: {player_count}") 

Total number of unique players: 576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [143]:
#Create calculations and a dataframe to display the total purchases analysis
uniqueItems = df_pd["Item ID"].nunique()
items_sold = df_pd["Purchase ID"].count()
totalRev = df_pd["Price"].sum()
avgPrchs = round(df_pd["Price"].mean(),2)

#Eliminate Duplicate Items before calculating average item price
t_item_analysis = df_pd[['Item Name', 'Price']]
t_item_analysis = t_item_analysis.sort_values('Item Name')
t_item_analysis = t_item_analysis.drop_duplicates()

#average price of items after duplicates removed to get actual item avg price
avgPrice = round(t_item_analysis["Price"].mean(),2)

purchase_summary = pd.DataFrame({"Unique Items": uniqueItems, "Average Item Price":"${:}".format(avgPrice), "Total Items Sold":items_sold
                                 ,"Average Spent on Items":"${:}".format(avgPrchs),"Total Revenue":"${:}".format(totalRev)},index=["Data"])
purchase_summary.head()

,Unique Items,Average Item Price,Total Items Sold,Average Spent on Items,Total Revenue
Data,183,$3.04,780,$3.05,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [136]:
df_players = df_pd[['SN', 'Age','Gender']]
df_players = df_players.sort_values('SN')
df_players = df_players.drop_duplicates()
male_players = df_players.loc[df_players["Gender"] == "Male","Gender"].count()
female_players = df_players.loc[df_players["Gender"] == "Female", "Gender"].count()
nd_players = df_players.loc[df_players["Gender"] == "Other / Non-Disclosed", "Gender"].count()

male_percent = round(male_players/player_count, 4) * 100
female_percent = round(female_players/player_count, 4) * 100
nd_percent = round(nd_players/player_count, 4) * 100

gender_demo = pd.DataFrame({"Total Players": [player_count], "Male Players":[male_players], "Male Percent":[male_percent]
                            , "Female Players":[female_players], "Female Percent":[female_percent]
                            , "Non-Disclosed":[nd_players], "Non-Disclosed Percent":[nd_percent]})
gender_demo.head()

,Total Players,Male Players,Male Percent,Female Players,Female Percent,Non-Disclosed,Non-Disclosed Percent
0,576,484,84.03,81,14.06,11,1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [156]:
#create data frame for non-duplicated player counts
gndr_players = df_pd[['SN','Gender']]
gndr_players = gndr_players.sort_values('SN')
gndr_players = gndr_players.drop_duplicates()

#group the distinct user dataframe by gender and rename the column
gndr_join = pd.DataFrame(gndr_players.groupby(["Gender"]).count())
gndr_join = gndr_join.rename(columns = {"SN":"Unique Players"})

#Create data frame for gender/price calcualtions to join with non-dup counts
gndr_sales2 = pd.DataFrame(df_pd[['Gender','Price']])
gndr_sales3 = gndr_sales2.rename(columns = {"Price":"Purchase Count"})

#calculations for gender analysis
gndr_count = pd.DataFrame(gndr_sales3.groupby(["Gender"]).count())
gndr_count["Total Purchase Value"]= gndr_sales3.groupby(["Gender"]).sum()
gndr_count["Average Purchase Price"] = round(gndr_sales3.groupby(["Gender"]).mean(), 2)
gndr_count["% Of Sales"] = round(gndr_count["Purchase Count"] / len(gndr_sales2), 4) * 100
gndr_count["% of Total Rev"] = round(gndr_count["Total Purchase Value"] / totalRev, 4) * 100

#join the data frames for the last calculation
gndr_join_sum = gndr_join.join(gndr_count)

gndr_join_sum["Avg Total Purchase per Person"] = round(gndr_join_sum["Total Purchase Value"] / gndr_join_sum["Unique Players"],2)

gndr_join_sum.head()
#gndr_count.describe()

,Unique Players,Purchase Count,Total Purchase Value,Average Purchase Price,% Of Sales,% of Total Rev,Avg Total Purchase per Person
Gender,,,,,,,
Female,81,113,361.94,3.20,14.49,15.21,4.47
Male,484,652,1967.64,3.02,83.59,82.68,4.07
Other / Non-Disclosed,11,15,50.19,3.35,1.92,2.11,4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [158]:
bins = [0, 9, 14, 19,24,29,34,39,200]

age_groups = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

ages = df_pd[["SN","Age"]]
ages = ages.drop_duplicates(["SN"])

ages["Age Groups"] = pd.cut(ages["Age"], bins, labels = age_groups)
ages = ages[["Age Groups","Age"]]            
age_group = ages.groupby(["Age Groups"]).count()
age_group = age_group.rename(columns = {"Age":"Total Count"})
age_group["Percentage of Players"] = round((age_group["Total Count"] / player_count) * 100,2)


age_group

,Total Count,Percentage of Players
Age Groups,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [125]:
#create age groups and bin the data to summarize purchases by age groups

bins2 = [0, 9, 14, 19, 24, 29, 34, 39, 200]

age_groups2 = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

ages_prchs = pd.DataFrame(df_pd[["SN","Age", "Price"]])
ages_prchs.head()

ages_prchs["Age Groups"] = pd.cut(ages_prchs["Age"], bins2, labels = age_groups2)

ages_prchs_g = ages_prchs.groupby(["Age Groups"]).count()
ages_prchs_g = ages_prchs_g.rename(columns = {"Age":"Purchase Count"})

ages_prchs_p = ages_prchs.groupby(["Age Groups"]).sum()
ages_prchs_p = ages_prchs_p.rename(columns = {"Price":"Total Purchase Value"})

ages_prchs_m = round(ages_prchs.groupby(["Age Groups"]).mean(),2)
ages_prchs_m = ages_prchs_m.rename(columns = {"Price":"Average Purchase Price"})

merge1_ap = ages_prchs_g.merge(ages_prchs_p, how = "inner", on = "Age Groups")
merge2_ap = merge1_ap.merge(ages_prchs_m, how = "inner", on = "Age Groups")


age_prchs_sum = merge2_ap[["Purchase Count", "Total Purchase Value", "Average Purchase Price"]]

age_prchs_sum["Avg Total Purchase per Person"] = round(age_prchs_sum["Total Purchase Value"] / age_group["Total Count"],2)
    
age_prchs_sum

C:\Users\bendgame\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
Age Groups,,,,
<10,23,77.13,3.35,4.54
10-14,28,82.78,2.96,3.76
15-19,136,412.89,3.04,3.86
20-24,365,1114.06,3.05,4.32
25-29,101,293.00,2.90,3.81
30-34,73,214.00,2.93,4.12
35-39,41,147.67,3.60,4.76
40+,13,38.24,2.94,3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [126]:
#Create data frames to calculate the purchase data per user and merge them to form a summary data frame.

#
top_spenders = pd.DataFrame(df_pd[["SN","Item ID", "Price"]])

top_spenders_g = top_spenders.groupby("SN").count()
top_spenders_g = top_spenders_g.rename(columns = {"Item ID":"Purchase Count"})

top_spenders_m = round(top_spenders.groupby("SN").mean(),2)
top_spenders_m = top_spenders_m.rename(columns = {"Price":"Average Purchase Price"})

top_spenders_t = top_spenders.groupby("SN").sum()
top_spenders_t = top_spenders_t.rename(columns = {"Price":"Total Purchase Value"})

merge1_ts = top_spenders_g.merge(top_spenders_m, how = "inner", on = "SN")
merge2_ts = merge1_ts.merge(top_spenders_t, how = "inner", on = "SN")


age_prchs_sum = merge2_ts[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

age_prchs_sum = age_prchs_sum.sort_values("Total Purchase Value", ascending = False)

age_prchs_sum.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.79,18.96
Idastidru52,4,3.86,15.45
Chamjask73,3,4.61,13.83
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [127]:
popular_items = df_pd[["Item ID","Item Name", "Price"]]
popular_items = popular_items.rename(columns = {"Price":"Purchase Count"})
popular_items_g = popular_items.groupby(["Item ID", "Item Name"]).count()

popular_items_g["Item Price"] = popular_items.groupby(["Item ID", "Item Name"]).mean()
popular_items_g["Total Purchase Value"] = popular_items.groupby(["Item ID", "Item Name"]).sum()

popular_items_g = popular_items_g.sort_values("Purchase Count", ascending = False)
popular_items_g.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
82,Nirvana,9,4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [164]:
profitable_items_g = popular_items_g.sort_values("Total Purchase Value", ascending = False)
profitable_items_g["Total Purchase Value"] = profitable_items_g["Total Purchase Value"].map("${:.2f}".format)
profitable_items_g.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,$50.76
82,Nirvana,9,4.90,$44.10
145,Fiery Glass Crusader,9,4.58,$41.22
92,Final Critic,8,4.88,$39.04
103,Singed Scalpel,8,4.35,$34.80
